# **Advanced Lane Finding**

## **Udacity Self Driving Car Engineer Nanodegree - Project 2**

The goal of this project is to develop a pipeline to process a video stream from a forward-facing camera mounted on the front of a car, and output an annotated video which identifies:

* The positions of the lane lines
* The location of the vehicle relative to the center of the lane
* The radius of curvature of the road

This can be done using advanced computer vision and colorspace exploration. The important steps of the project include -

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

## Description:

### Step 1: Camera calibration and Image Undistortion

The first step in the project is to remove any distortion from the images by calculating the camera calibration matrix and distortion coefficients using a series of images of a chessboard.


#### * Camera Calibration using chessboard images
#### * Image undistortion using image points and object points obtained 

Since opencv function calibrateCamera only works on grayscale images, so first convert the image to grayscale and use it to calculate undistortion matrix for conversion. Using this matrix we can undistort any colorspace image.

<figure>
 <img src="test_images_output/undistortion.jpg" width="800" alt="Combined Image" />
</figure>

The difference is very subtle to note, but it exists.

### Step 2: Keypoint detection

Keypoint detection using opencv SIFT helps in calculating the points og contrast and color differences i.e corners in an image. I tried using keypoint detection to identify the top and bottom of lane lines and then use them to warp the image perspective. 

However, this increases the computation of the pipeline and with ambiguous goals. We can use the hyperparameters for warping the image and it does a good task. 

However, on a road with very steep elevation the hyperparameter fitting might fail to gve good results, so this can be a good substitute way to find the points required for perspective transform.

<figure>
 <img src="test_images_output/Keypoint_img.jpg" width="800" alt="Combined Image" />
</figure>


### Step 3: Perspective transform

Perspective transform is used to change the perspective or viewpoint of an image. OpenCV provides fuctions for this purpose.

Here we want to change the image perspective from front view to a bird's eye view so that the features of lane line can be read easily.

As per the requirement of image processing we need to transform the image such that it covers only the lane area of image. For this we need 4 points on the original image and where we want them to be in the warped view. I used:

```python
# define source and destination points for transform
    src = np.float32([(575,464),
                      (707,464), 
                      (258,682), 
                      (1049,682)])
    dst = np.float32([(450,0),
                      (w-450,0),
                      (450,h),
                      (w-450,h)])
```
where w and h are width = 720 and height = 1280 of image.

<figure>
 <img src="test_images_output/perspective.jpg" width="800" alt="Combined Image" />
</figure>


### Step 4: Color masking and scale conversion

This step is implemented to remove all the noise from an image using different color channels and color thresholds and just keep the relevant information intact in the image (in this case the lane lines). Following steps were taken :

#### RGB to HLS Conversion

RGB to HLS conversion of an image helped in the previous project on lane finding, so tried it. Does not give good results on light condition variances in this project. However, L-channel of HLS colorspace within the range `[220,255]` reads white lines precisely with varying light conditions.

<figure>
 <img src="test_images_output/hls_image_thresholded.jpg" width="800" alt="Combined Image" />
</figure>


#### RGB to LAB Conversion

LAB is another such color space with the B-channel within the range `[190,255]` detects white lines almost accurately.

<figure>
 <img src="test_images_output/lab_image_thresholded.jpg" width="800" alt="Combined Image" />
</figure>


#### Other conversions

HSV, YCrCb colorspces were also tested for image masking. Best fit was obtained using the above 2 channels. Others were discarded.

*White Yellow Thresholding*

<figure>
 <img src="test_images_output/wy_image_thresholded.jpg" width="800" alt="Combined Image" />
</figure>

*YCrCb Image*

<figure>
 <img src="test_images_output/ycrcb_image.jpg" width="800" alt="Combined Image" />
</figure>

*HLS Image*

<figure>
 <img src="test_images_output/hls_image.jpg" width="800" alt="Combined Image" />
</figure>

*HSV Image*

<figure>
 <img src="test_images_output/hsv_image.jpg" width="800" alt="Combined Image" />
</figure>

*LAB Image*

<figure>
 <img src="test_images_output/lab_image.jpg" width="800" alt="Combined Image" />
</figure>


#### Final Conversion

Results after white and yellow color masking using HLS-L and LAB-B channels is visualized.

<figure>
 <img src="test_images_output/hls_lab_combination.jpg" width="800" alt="Combined Image" />
</figure>


### Step 5: GrayScale conversion

This conversion is pretty good for gradient calculation and edge detections. But I didnot use it later in my pipeline because my image was already single channel binary format.

<figure>
 <img src="test_images_output/grayscaled.jpg" width="800" alt="Combined Image" />
</figure>

### Step 6: Sobel transformation

Sobel transform of combined_img. Sobel Magnitude threshold and Sobel Gradient threshold were considered to efficiently remove noise. I implemented sobel transform in following way:

```python
    #sobel x and y transforms
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize= kernel)
    sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize= kernel)
    # absolute value of transforms
    abs_sobelx = np.absolute(sobelx)
    abs_sobely = np.absolute(sobely)
    # magnitude of transforms
    mag_sobel = np.sqrt(abs_sobelx**2 + abs_sobely**2)
    # direction of transforms
    dir_sobel = np.arctan2(abs_sobely,abs_sobelx)
    #magnitude thresholding
    mag_thres_sobel = np.zeros_like(mag_scaled_sobel)
    mag_thres_sobel[(mag_scaled_sobel >= mag_threshold[0]) & (mag_scaled_sobel < mag_threshold[1])] = 1
    #direction thresholding
    dir_thres_sobel = np.zeros_like(dir_sobel)
    dir_thres_sobel[(dir_sobel >= dir_threshold[0]) & (dir_sobel < dir_threshold[1])] = 1
    #combining 2 thresholdings
    final_sobel = np.zeros_like(img)
    final_sobel[(mag_thres_sobel == 1) & (dir_thres_sobel == 1)] = 1
   ```
   
   where threshold for magnitude was set `[20,255]` and threshold for direction was set `[0,0.5]` . /

<figure>
 <img src="test_images_output/sobel_transformed.jpg" width="800" alt="Combined Image" />
</figure>

### Step 7: Sliding window polyfit for Lane find

At this point I was able to use the combined binary image to isolate only the pixels belonging to lane lines. The next step was to fit a polynomial to each lane line, which was done by:

* Identifying peaks in a histogram of the image to determine location of lane lines.
* Identifying all non zero pixels around histogram peaks using the numpy function numpy.nonzero().
* Fitting a polynomial to each lane using the numpy function numpy.polyfit().

This can be done by sliding window polyfit method where you select the non-zero pixel without knowing the previous location of lanes. 

<figure>
 <img src="test_images_output/sliding_window.jpg" width="800" alt="Combined Image" />
</figure>

### Step 8: Previous frame usage

If you know the previous loaction of lane lines a search within a given area around those lanes helps in finding lines continuously and computationally faster than rectangular area search.

<figure>
 <img src="test_images_output/previous_polyfit.jpg" width="800" alt="Combined Image" />
</figure>

### Step 9: Radius of Curvature and Distance from center of Lane calculation

After fitting the polynomials I was able to calculate the radius of curvature and position of the vehicle with respect to center with the following calculations:

* Calculated the radius of curvature of left and right lane given polynomial coefficients = (1+(2A+B)^2)^1.5)/|2A|  where A. B are coefficients of equation Ax^2 + Bx + C 
* Calculated lane curvature as average curvature of both lanes.
* Calculated the average of the x intercepts from each of the two polynomials position = (rightx_int+leftx_int)/2
* Calculated the distance from center by taking the absolute value of the vehicle position minus the halfway point along the horizontal axis distance_from_center = abs(image_width/2 - position)
* If the horizontal position of the car was greater than image_width/2 than the car was considered to be left of center, otherwise right of center.
* Finally, the distance from center was converted from pixels to meters by multiplying the number of pixels by 3.7/400 in x-direction and 3.048/100 in y-direction.

### Step 10: Inverting Perspective and marking Lanes on original image

It's better to mark the lanes on the warped image and create a mask of lmarked up lane, and then change the perspective of the mask using perspective transform inverse matrix and add it to the original image.

<figure>
 <img src="test_images_output/original_image_marked.jpg" width="800" alt="Combined Image" />
</figure>

### Step 11: Write Radius of Curvature and Distance from Center on image

<figure>
 <img src="test_images_output/final_image.jpg" width="800" alt="Combined Image" />
</figure>

### Step 12: Define class for saving lane data

I created a class for saving the data like recent_fit, best_fit and current_fit of a lane in the following way:

```python
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None
        #polynomial coefficients of the last n fits of the line
        self.recent_fit = []
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = []  
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 

        
    def add_fit_coeff(self, fit):
        if fit is not None and fit is not [0,0,450] and fit is not [0,0,830]:
            if self.best_fit is not None:
                self.diffs = np.absolute(fit - self.best_fit)
            if self.diffs[0] > 0.01 or self.diffs[1] > 1 or self.diffs[2] > 100 :
                self.detected = False
                if len(self.recent_fit) > 0:
                    self.current_fit = self.recent_fit[len(self.recent_fit)-1]
                else:
                    self.current_fit = fit
            else:
                self.detected = True
                self.current_fit = fit
                if len(self.recent_fit) != 0:
                    if len(self.recent_fit) > 5:
                        self.recent_fit = np.append(self.recent_fit,[fit], axis =0)
                        self.recent_fit = self.recent_fit[len(self.recent_fit)-5:]
                    else:
                        self.recent_fit = np.append(self.recent_fit,[fit], axis =0)
                    self.best_fit = np.average(self.recent_fit, axis=0)
                else:
                    self.recent_fit = [fit]
        
        else:
            self.detected = False
            if len(self.recent_fit) > 0:
                # throw out oldest fit
                self.recent_fit = self.recent_fit[1:len(self.recent_fit)]
            if len(self.recent_fit) > 0:
                # if there are still any fits in the queue, best_fit is their average
                self.current_fit = np.average(self.recent_fit, axis=0)
                self.best_fit = np.average(self.recent_fit, axis=0)
            elif self.best_fit is not None:
                self.current_fit = self.best_fit
            else:
                self.current_fit = None

        
    def add_xvalues(self, values):
        if len(self.recent_xfitted) != 0:
            if len(self.recent_xfitted)>5:
                self.recent_xfitted = np.hstack([self.recent_xfitted, values])
                self.recent_xfitted = self.current_fit[len(self.current_fit)-5:]
            else:
                self.recent_xfitted = np.hstack([self.recent_xfitted, values])
            self.bestx = np.average(self.recent_xfitted, axis=0)
            
        else:
            self.recent_xfitted = values
        
    ```
    
    I also added curve radius and lane positions to the class but, saving it and using the data for future purpose gives somewhat the same result as without them. So I dropped it. However, it can be added if you want to run diagnostics on the data you had and how your pipeline processed it. 
    
### Step 13: Final Pipeline for image processing

```python
objpoints , imgpoints = calibration_point()

#final pipeline for lane finding
def pipeline(img):
    original_img = np.copy(img)
    binary_img, Minv = preprocessing_pipeline(img)
    
    #if lanes were detected in previous frame, then use search_around_poly else, use sliding_window_polyfit_generator
    if not left.detected or not right.detected :
        left_fit, right_fit = sliding_window_polynomial(binary_img)
    else:
        left_fit, right_fit = search_around_polynomial(binary_img, left.current_fit, right.current_fit)
    
    # invalidate both fits if the difference in their x-intercepts isn't around 350 px (+/- 100 px)
    if left_fit is not None and right_fit is not None:
        # calculate x-intercept (bottom of image, x=image_height) for fits
        h = img.shape[0]
        left_fit_x = left_fit[0]*h**2 + left_fit[1]*h + left_fit[2]
        right_fit_x = right_fit[0]*h**2 + right_fit[1]*h + right_fit[2]
        bottom_x_diff = abs(right_fit_x-left_fit_x)
        if abs(350 - bottom_x_diff) > 100:
            left_fit = None
            right_fit = None
            
            
    left.add_fit_coeff(left_fit)
    right.add_fit_coeff(right_fit)
    
    # draw the current fit if it exists
    
    img_out1 = original_image_lane_marker(original_img, binary_img, left.current_fit, right.current_fit, Minv)
    radius, center, left_fitx, right_fitx = radius_central_distance_calculator(img, left.current_fit, right.current_fit)
    left.add_xvalues(left_fitx)
    right.add_xvalues(right_fitx)
    img_out = image_writer(img_out1, radius, center)
    return img_out
                ```
                
                
## Results on Video Feeds

Video feeds for the 3 challenges given in the project are linked below. Drawing lines throughout snaps of the video wherever the pipeline returns a marked up image helps in creating lane lines on the video as a whole. This can be used for finding lane lines in an online fashion.

[Project Video](project_video_output.mp4)

[Challenge Video](challenge_video_output.mp4)

[Harder Challenge Video](harder_challenge_video_output.mp4)

## Potential Shortcomings

1. The pipeline fails to detect lanes when the lighting condition varies. Say in case there is a shadow of a tree or a flyover overhead, both cause issues with perspective transform and hence hampers the pipeline on the whole.

2. The pipeline fails to detect lanes when the lane lines are changing fast, say you are driving through a mountaneous terrain and lanes curve left and right in matter of seconds. The piepline fails in such case as can be seen in harder_challenge_video.

3. We have assumed a transformation matrix in perspective transform using predefined set of 4 points which is approximately the entire length of road given the elevation of road is horizontal. However, when the elevation of road varies significantly perspective transform will not be accurate and thus lane finding will be hampered.

4. What is there is no lane line on roads, say streets. You cant let the car drive on its own in a small locality.


## Possible Improvements

1. Dynamic region selection based on gradient of road can be implemented to only keep lane lines in region of interest for the pipeline.

2. Advanced algorithms to deal with curves along the road should be implemented to make the pipeline more generic.

3. Better way to use computer vision to detect lanes based on color selection even with varied light conditions can be used if they exist.

4. Advanced sensors like radars, lidars and sensor fusion can be used to cover a large area of conditional states during driving a car. 